# From coverage files to differential accessibility

Check what is your working directory.

In [ ]:
getwd()

## Load your sample plan

In [ ]:
splan <-read.csv("data/metadata.tsv", sep="\t",row.names=1, header=TRUE)
splan

## Read coverage files

In [ ]:
list.files('output/peak_coverage')

Get the path to the individual peak files. 

In [ ]:
exprs.in <-lapply(rownames(splan), function(x) { paste("output/peak_coverage",'/',x,'_peaks.bed',sep="")}) 

Then read the peak file.  

Column description:
1. chr 
2. start 
3. end 
4. peak_ID 
5. MACS_score  
6. MACS_strand 
7. MACS_fold_enrichment 
8. MACS_-log10(pval) 
9. MACS_-log10(qval) 
10. MACS_summit (0-based offset from chrom start)
11. reads_in_peak 
12. peak_non-zero_bases 
13. peak_length 
14. peak_fraction_covered

In [ ]:
counts.exprs <- lapply(exprs.in, read.csv, sep="\t", header=FALSE, check.names=FALSE, col.names=c('chr','start','end', 'peak','score','strand','fold_enr','-log10(pval)','-log10(qval)','summit','reads_in_peak','non-zero_bases','peak_lenght','cov_fraction'))

## Combine the data into a single count table, using peak coordinates as an ID.
edgeR works on a table of integer read counts, with rows corresponding to regions and columns to independent libraries. 

In [ ]:
i <- 0
for (sample in counts.exprs) {
    if (i==0) {
        sample$ID <- paste(sample$chr,sample$start,sample$end,sep="_")
        count_table <- data.frame(as.numeric(sample$reads_in_peak), row.names=sample$ID)
        }
    else {count_table <- data.frame(count_table,as.numeric(sample$reads_in_peak))}
    i <- i+1
}
## rename the columns
names(count_table) <- rownames(splan)
head(count_table)

In [ ]:
message("size of the table")
message("number of ATAC peaks")
message(dim(count_table)[1])
message("number of samples")
message(dim(count_table)[2])
message("number of counts per sample")
print(colSums(count_table),na.rm = TRUE)

## Install / load edgeR

In [ ]:
## if necessary, install edgeR 
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("edgeR")

In [ ]:
# load the edgeR package
library(edgeR)

## Create a DGE object and normalise the counts

edgeR stores data in a simple list-based data object called a DGEList. The groups are defined in the 2nd columns of our sample plan. 

In [ ]:
y <- DGEList(counts=count_table,group=splan$group)

The calcNormFactors() function normalizes for reads composition by finding a set of scaling factors for the library sizes that minimize the log-fold changes between the samples for most regions. The default method for computing these scale factors uses a trimmed mean of M-values (TMM) between each pair of samples. The effective library size replaces the original library size in all downsteam analyses.

In [ ]:
y <- calcNormFactors(y)

Check that the design is correct and see the normalisation factors. 

In [ ]:
y$samples

## Data Exploration
Before proceeding with the computations for differential accessibility, it is possible to produce a plot showing the sample relations based on multidimensional scaling. 

In [ ]:
plotMDS(y, col=as.numeric(y$samples$group))
legend("bottomleft", as.character(unique(y$samples$group)), col=1:3, pch=20)

## Estimating the Dispersion

In [ ]:
y <- estimateCommonDisp(y,verbose=T)

In [ ]:
y <- estimateTrendedDisp(y)
y <- estimateTagwiseDisp(y)
plotBCV(y)

# <span style="color:red">TODO, read the doc to check what to do with such dispersion...</span>.

## Differential accessibility
Once the dispersions are estimated, we can proceed with testing procedures for determining differential accessibility. The function exactTest() conducts tagwise tests using the exact negative binomial test. The test results for the n most significant tags are conveniently displayed by the topTags() function. By default, Benjamini and Hochberg's algorithm is used to control the false discovery rate (FDR).

In [ ]:
et <- exactTest(y)
topTags(et)

The number of differentially accessible regions at FDR< 0:05 is:

In [ ]:
Z=summary(de <- decideTestsDGE(et, p.value=0.05, adjust="BH"))
Z

The function plotSmear generates a plot of the tagwise log-fold-changes against log-cpm. Differential tags are highlighted on the plot. 

In [ ]:
detags <- rownames(y)[as.logical(de)]
plotSmear(et, ylim=c(-6,6), de.tags=detags)
abline(h = c(-2, 2), col = "blue")

## Export the differential analysis results

In [ ]:
et.df = as.data.frame(do.call(rbind, et))
write.table(et.df,"differential_accessibility.tsv",quote=FALSE,row.names=TRUE,sep="\t",col.names=NA)

# <span style="color:red">TODO, read the doc to check if GLM would be more appropriate </span>.